In [1]:
OUT_DIR = '/tmp/'

from torchvision import models
import torchvision.datasets as dsets
import json
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR, ImageNet
import torch
from cox.utils import Parameters
import cox.store
from torchvision import models
import torch
from torchvision import transforms
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
# Standard libraries
import numpy as np
import os
# PyTorch
import torch.nn as nn
from PIL import Image

/home/hashmat/miniconda3/envs/advdrop_env/lib/python3.7/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [2]:

class_idx = json.load(open("/home/hashmat/Thesis/AdvDrop/imagenet_class_index.json"))
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
class2label = [class_idx[str(k)][0] for k in range(len(class_idx))]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), ])


def image_folder_custom_label(root, transform, idx2label) :
    
    
    old_data = dsets.ImageFolder(root=root, transform=transform)
    old_classes = old_data.classes
    
    label2idx = {}
    
    for i, item in enumerate(idx2label) :
        label2idx[item] = i
    
    new_data = dsets.ImageFolder(root=root, transform=transform, 
                                 target_transform=lambda x : idx2label.index(old_classes[x]))
    new_data.classes = idx2label
    new_data.class_to_idx = label2idx

    return new_data

def create_dir(dir, print_flag = False):
    if not os.path.exists(dir):
        os.mkdir(dir)
        if print_flag:
            print("Create dir {} successfully!".format(dir))
    elif print_flag:
        print("Directory {} is already existed. ".format(dir))


def get_data(data_dir, transform, batch_size, class2label):

    normal_data = image_folder_custom_label(root=data_dir,
                                            transform=transform,
                                            idx2label=class2label)
    normal_loader = torch.utils.data.DataLoader(normal_data,
                                                batch_size=batch_size,
                                                shuffle=False)

    return normal_loader

## Load VGG Models

In [3]:
imagenet_ds = ImageNet('/tmp/')
vgg , _ = model_utils.make_and_restore_model(arch=models.vgg16_bn(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = vgg.model

=> loading checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt'
=> loaded checkpoint '/home/hashmat/Thesis/AdvDrop/pretrained-models/vgg16_bn_l2_eps0.ckpt' (epoch 89)


## Load MobileNet-v2

In [ ]:
imagenet_ds = ImageNet('/tmp/')
mobilenet , _ = model_utils.make_and_restore_model(arch=models.mobilenet_v2(), dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/mobilenet_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = mobilenet.model

## Load ResNet Models

In [ ]:
imagenet_ds = ImageNet('/tmp/')
resnet , _ = model_utils.make_and_restore_model(arch="resnet18", dataset=imagenet_ds, 
                                resume_path='/home/hashmat/Thesis/AdvDrop/pretrained-models/resnet18_l2_eps0.ckpt',
                                parallel=False, add_custom_forward=True)
model = resnet.model

In [4]:
#from Models.transformers import diet_tiny, diet_small, vit_tiny, vit_small
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))

    def forward(self, input):
        # Broadcasting
        input = input / 255.0
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std

    transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])])

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

OUT_DIR = '/tmp/'
imagenet_ds = ImageNet('/tmp/')




model = model.eval()
batch_size = 4

data_dir = "/home/hashmat/Thesis/nobox-attacks/data/ILSVRC2012_img_val"
normal_loader = get_data(data_dir, transform, batch_size, class2label)


correct = 0
total = 0
count = 0
for images, labels in normal_loader:
    print(count)
    count+=1
    images, labels = images.to(device), labels.to(device)
    preds = model(images)

    _, pre = torch.max(preds, 1)
    correct += torch.sum(pre==labels)
    total += images.shape[0]
print(f"The Model accuracy is {correct/total}")

0
Here
1
2
3
4
5
6
7
Here
8
Here
9
10
11
12
13
14
15
Here
16
17
18
19
20
21
22
23
24
25
26
Here
27
28
29
Here
30
31
32
33
34
35
36
Here
37
38
39
40
41
Here
42
43
Here
44
45
46
47
48
49
Here
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
Here
79
80
81
82
83
Here
84
85
86
Here
87
88
89
Here
90
Here
91
92
Here
93
94
95
96
97
98
99
Here
100
101
102
103
104
Here
105
106
Here
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
Here
131
132
133
134
Here
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
Here
151
152
Here
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
Here
180
181
Here
182
Here
183
Here
184
185
186
187
Here
188
189
Here
190
191
192
Here
193
194
195
196
197
198
199
200
201
Here
202
203
204
205
Here
206
207
208
209
Here
210
211
212
213
214
215
216
217
Here
218
Here
219
220
221
222
223
224
225
226
227
228
229
Here
230
231
23

1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
Here
1520
1521
1522
1523
1524
Here
1525
1526
Here
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
Here
1538
1539
Here
1540
1541
1542
1543
1544
1545
1546
1547
1548
Here
1549
1550
1551
Here
1552
1553
1554
Here
1555
1556
Here
1557
1558
Here
1559
Here
1560
Here
1561
1562
Here
1563
1564
1565
1566
1567
1568
1569
Here
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
Here
1581
1582
1583
1584
1585
1586
1587
1588
Here
1589
1590
1591
1592
1593
1594
1595
Here
1596
1597
1598
Here
1599
1600
1601
1602
1603
1604
Here
1605
Here
1606
Here
1607
1608
1609
1610
Here
1611
1612
Here
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
Here
1640
1641
1642
1643
1644
Here
1645
Here
1646
Here
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
Here
1678
Here
1679


2784
2785
Here
2786
2787
2788
2789
2790
Here
2791
2792
2793
2794
2795
2796
2797
Here
2798
2799
2800
2801
Here
2802
2803
2804
2805
2806
Here
2807
Here
2808
2809
2810
2811
2812
Here
2813
2814
2815
Here
2816
2817
Here
2818
2819
2820
2821
Here
2822
2823
2824
Here
2825
2826
Here
2827
2828
2829
2830
2831
Here
2832
2833
Here
2834
Here
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
Here
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
Here
2870
2871
2872
Here
2873
2874
2875
2876
2877
Here
2878
Here
2879
Here
2880
Here
2881
Here
2882
Here
2883
2884
2885
Here
2886
Here
2887
2888
2889
2890
2891
2892
Here
2893
2894
2895
2896
Here
2897
2898
2899
2900
2901
2902
2903
Here
2904
2905
Here
2906
2907
2908
2909
Here
2910
2911
2912
2913
Here
2914
2915
2916
2917
Here
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
Here
2937
2938
2939
2940
2941
2942
2943
Here
2944
2945
2946
2947
2948


4119
Here
4120
4121
4122
Here
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
Here
4135
Here
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
Here
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
Here
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
Here
4191
4192
4193
4194
4195
Here
4196
4197
4198
Here
4199
Here
4200
4201
4202
4203
4204
4205
4206
4207
4208
4209
4210
4211
4212
4213
4214
4215
Here
4216
Here
4217
4218
Here
4219
4220
4221
4222
Here
4223
4224
4225
Here
4226
4227
Here
4228
4229
Here
4230
4231
4232
4233
Here
4234
4235
Here
4236
4237
4238
4239
4240
4241
4242
4243
4244
4245
4246
4247
4248
4249
4250
4251
4252
4253
4254
4255
4256
4257
4258
4259
4260
4261
4262
4263
4264
Here
4265
Here
4266
4267
4268
Here
4269
Here
4270
Here
4271
Here
4272
Here
4273
Here
4274
Here
4275
4276
Here
4277
4278
Here
4279
4280
4281
4282
Here
4283
Here
4284
Here
4285


5407
5408
5409
5410
5411
5412
5413
Here
5414
Here
5415
Here
5416
Here
5417
Here
5418
5419
Here
5420
5421
5422
5423
Here
5424
Here
5425
Here
5426
Here
5427
Here
5428
Here
5429
Here
5430
5431
Here
5432
5433
Here
5434
5435
Here
5436
5437
5438
5439
5440
5441
Here
5442
5443
Here
5444
Here
5445
Here
5446
5447
5448
5449
Here
5450
5451
5452
5453
Here
5454
5455
5456
5457
5458
5459
5460
Here
5461
Here
5462
5463
5464
5465
5466
5467
Here
5468
5469
Here
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
Here
5486
5487
5488
5489
Here
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
Here
5501
Here
5502
5503
Here
5504
Here
5505
Here
5506
Here
5507
Here
5508
Here
5509
5510
Here
5511
5512
5513
5514
5515
Here
5516
Here
5517
Here
5518
Here
5519
5520
Here
5521
Here
5522
5523
Here
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
Here
5535
Here
5536
5537
5538
Here
5539
5540
5541
5542
Here
5543
5544
5545
5546
Here
5547
5548
5549
Here
5550
Here
5551
5552
5553
5554
5555


...

7853
7854
7855
7856
7857
7858
7859
7860
7861
7862
7863
7864
7865
7866
7867
Here
7868
7869
Here
7870
7871
Here
7872
Here
7873
7874
7875
7876
7877
Here
7878
Here
7879
Here
7880
7881
Here
7882
7883
Here
7884
7885
7886
7887
7888
7889
7890
7891
7892
Here
7893
7894
7895
7896
Here
7897
Here
7898
7899
7900
7901
7902
7903
Here
7904
7905
7906
Here
7907
7908
Here
7909
7910
Here
7911
7912
7913
7914
7915
7916
Here
7917
Here
7918
Here
7919
7920
Here
7921
Here
7922
Here
7923
Here
7924
Here
7925
Here
7926
7927
7928
Here
7929
7930
Here
7931
Here
7932
7933
7934
Here
7935
7936
7937
Here
7938
7939
Here
7940
7941
Here
7942
Here
7943
7944
7945
Here
7946
7947
7948
7949
Here
7950
7951
Here
7952
7953
Here
7954
7955
Here
7956
Here
7957
Here
7958
7959
7960
Here
7961
7962
Here
7963
7964
7965
Here
7966
7967
Here
7968
Here
7969
7970
Here
7971
7972
Here
7973
Here
7974
7975
Here
7976
Here
7977
Here
7978
7979
Here
7980
7981
Here
7982
7983
Here
7984
Here
7985
7986
Here
7987
Here
7988
Here
7989
7990
Here
7991
7992
Here


9096
9097
9098
9099
9100
9101
Here
9102
Here
9103
Here
9104
Here
9105
Here
9106
9107
Here
9108
9109
Here
9110
Here
9111
Here
9112
9113
9114
9115
9116
9117
9118
9119
Here
9120
9121
9122
Here
9123
Here
9124
9125
9126
9127
9128
Here
9129
9130
Here
9131
9132
9133
9134
Here
9135
9136
9137
9138
Here
9139
Here
9140
9141
9142
Here
9143
Here
9144
9145
9146
Here
9147
9148
Here
9149
Here
9150
9151
9152
Here
9153
9154
9155
9156
Here
9157
9158
9159
9160
9161
9162
9163
Here
9164
Here
9165
Here
9166
Here
9167
Here
9168
9169
9170
Here
9171
9172
9173
Here
9174
Here
9175
9176
9177
9178
Here
9179
Here
9180
9181
9182
9183
9184
9185
9186
9187
9188
9189
Here
9190
9191
9192
9193
9194
9195
9196
Here
9197
Here
9198
9199
9200
9201
Here
9202
9203
Here
9204
9205
Here
9206
Here
9207
9208
9209
9210
9211
9212
9213
Here
9214
9215
Here
9216
9217
9218
9219
9220
9221
9222
9223
Here
9224
9225
9226
9227
9228
9229
9230
9231
9232
9233
9234
Here
9235
9236
9237
9238
9239
9240
9241
9242
9243
9244
9245
9246
9247
9248
9249
9250


10269
10270
10271
10272
10273
10274
10275
Here
10276
Here
10277
Here
10278
10279
Here
10280
10281
10282
10283
Here
10284
10285
Here
10286
10287
Here
10288
Here
10289
Here
10290
Here
10291
10292
Here
10293
Here
10294
10295
Here
10296
Here
10297
Here
10298
10299
10300
Here
10301
10302
Here
10303
Here
10304
10305
10306
10307
Here
10308
10309
10310
Here
10311
Here
10312
10313
Here
10314
10315
Here
10316
10317
Here
10318
Here
10319
10320
Here
10321
10322
10323
Here
10324
Here
10325
Here
10326
10327
Here
10328
Here
10329
Here
10330
10331
10332
10333
Here
10334
Here
10335
Here
10336
10337
10338
Here
10339
Here
10340
Here
10341
Here
10342
Here
10343
Here
10344
Here
10345
10346
Here
10347
Here
10348
Here
10349
Here
10350
10351
10352
Here
10353
Here
10354
Here
10355
10356
10357
10358
Here
10359
10360
Here
10361
10362
10363
10364
10365
10366
10367
10368
10369
10370
10371
10372
10373
10374
10375
10376
10377
Here
10378
10379
10380
Here
10381
Here
10382
Here
10383
Here
10384
10385
Here
10386
Here
10

Here
11329
Here
11330
Here
11331
Here
11332
Here
11333
Here
11334
11335
Here
11336
11337
Here
11338
Here
11339
Here
11340
Here
11341
Here
11342
Here
11343
Here
11344
Here
11345
Here
11346
Here
11347
Here
11348
Here
11349
Here
11350
11351
11352
Here
11353
Here
11354
11355
11356
Here
11357
11358
11359
Here
11360
Here
11361
Here
11362
11363
Here
11364
Here
11365
11366
Here
11367
Here
11368
Here
11369
Here
11370
Here
11371
Here
11372
Here
11373
Here
11374
Here
11375
11376
Here
11377
Here
11378
11379
11380
Here
11381
Here
11382
Here
11383
Here
11384
Here
11385
11386
Here
11387
Here
11388
Here
11389
Here
11390
Here
11391
Here
11392
Here
11393
Here
11394
11395
11396
11397
11398
11399
11400
11401
11402
11403
11404
11405
Here
11406
11407
11408
11409
11410
11411
Here
11412
11413
11414
Here
11415
11416
11417
11418
11419
11420
11421
11422
Here
11423
11424
11425
11426
11427
Here
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
1

12397
12398
12399
12400
Here
12401
12402
Here
12403
12404
12405
12406
12407
12408
12409
12410
12411
12412
12413
12414
12415
12416
12417
12418
Here
12419
12420
12421
12422
12423
12424
12425
Here
12426
12427
12428
Here
12429
12430
12431
12432
12433
12434
12435
12436
Here
12437
12438
12439
12440
12441
12442
12443
12444
12445
12446
12447
12448
12449
12450
12451
Here
12452
Here
12453
Here
12454
Here
12455
12456
12457
12458
Here
12459
12460
12461
12462
12463
12464
Here
12465
Here
12466
Here
12467
12468
12469
12470
12471
12472
Here
12473
Here
12474
12475
12476
12477
12478
Here
12479
Here
12480
12481
12482
12483
12484
Here
12485
Here
12486
12487
Here
12488
12489
Here
12490
Here
12491
Here
12492
Here
12493
Here
12494
Here
12495
12496
12497
12498
Here
12499
Here
The Model accuracy is 0.539359986782074
